In [204]:
import requests
import json
import pymongo
from pymongo import MongoClient
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import gensim
# from bson.objectid import ObjectId

# Connect to db

In [205]:
f = open("password.txt", "r")
pwd =  f.read()
f.close()

In [206]:
db_username = 'sveta'
db_password = pwd

In [207]:
db = MongoClient("mongodb+srv://" + db_username + ":" + db_password + "@makhela-qvsh8.mongodb.net/Makhela")

In [208]:
mydb = db["Makhela"]

In [209]:
mydb.list_collection_names()

['keywords', 'posts', 'opinion_leaders']

# Opinion Leaders

In [210]:
mycol = mydb["opinion_leaders"].find()

In [211]:
leaders = {}

for leader in mycol:
    try:
        leaders[leader["twitter_id"]] = leader
        leader["posts"] = []
#         print(leader["twitter_id"])
    except:
        continue

In [212]:
leaders[2696666707]

{'_id': ObjectId('5e4fe236f3d95c3a7b5f3b9f'),
 'full_name': 'Dalia Dassa Kaye',
 'twitter_id': 2696666707,
 'twitter_screen_name': 'dassakaye',
 'twitter_created_at': datetime.datetime(2014, 7, 31, 21, 24, 12),
 'twitter_description': 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation',
 'twitter_followers_count': 3489,
 'twitter_friends_count': 403,
 'twitter_location': 'Santa Monica, CA',
 'new_leader': False,
 'twitter_statuses_count': 3660,
 'level_of_certainty': 2,
 'twitter_profile_image': 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg',
 'community_following': [30181014,
  22803302,
  2759399473,
  1227326168,
  2190601093,
  43380129,
  1515812534,
  482120066,
  2739262472,
  1387074108,
  376435807,
  49702592,
  42176166,
  47156708,
  475454826,
  48906364,
  480678403,
  1114663532,
  488772327,
  61051322,
  541858751],
 'lock': False,
 'posts': []}

# Posts

In [213]:
mycol = mydb["posts"].find()

In [214]:
posts = {}

for post in mycol:
    try:
        posts[post["post_id"]] = post
    except:
        continue

In [215]:
posts[1235946337331343366]

{'_id': ObjectId('5e6407df124e74ea22f2a87e'),
 'leader_twitter_id': 2696666707,
 'post_id': 1235946337331343366,
 'full_text': '@michaeltanchum Yep, this applies globally. The need for truth, data and transparency as well.',
 'date_created': datetime.datetime(2020, 3, 6, 15, 12, 24),
 'in_reply_to_status_id': 1235945439180054528,
 'in_reply_to_status_text': '@dassakaye Exactly. Never has made it more clear in the U.S. that good governance is a matter of life and death.',
 'in_reply_to_status_user_id': 'michaeltanchum',
 'quoted_status_id': 'None',
 'quoted_status_text': 'None',
 'quoted_status_user_id': 'None',
 'retweeted_status_id': 'None',
 'retweeted_status_text': 'None',
 'retweeted_status_user_id': 'None',
 'checked_for_suggestions': False}

In [216]:
def text(dict):
    if dict['retweeted_status_text'] != 'None':
        return dict['retweeted_status_text']
    elif dict['in_reply_to_status_text'] != 'None':
        return dict['in_reply_to_status_text']
    elif dict['quoted_status_text'] != 'None':
        return dict['quoted_status_text']
    else:
        return dict['full_text']

In [217]:
print(text(posts[1235944924853317632]))

LEBANON #Coronavirus Crisis:  #COVID19 spreading in a country that has not been able to buy medical supplies for 5 months. Now Lebanon's health minister says the country is no longer focused on containing the virus https://t.co/NLSSH1PSMT


In [218]:
words_from_posts = []
for key, value in posts.items():
    try:
        post_row = []
        post_row.append(key)
        post_row.append(text(posts[key])) 
#         words_from_posts.append(text(posts[key])) 
        words_from_posts.append(post_row)
    except:
        continue

In [219]:
pr = pd.DataFrame(words_from_posts)
pr.head()

,0,1
0,1235946337331343366,@dassakaye Exactly. Never has made it more cle...
1,1235944924853317632,LEBANON #Coronavirus Crisis: #COVID19 spreadi...
2,1235938139119562754,A further isolated and weakened Iran will like...
3,1235161759423614977,"Coronavirus: Iran temporarily frees 54,000 pri..."
4,1234901226807939072,@dassakaye Not necessarily to be honest. #Saud...


In [220]:
pr[1].head()

0    @dassakaye Exactly. Never has made it more cle...
1    LEBANON #Coronavirus Crisis:  #COVID19 spreadi...
2    A further isolated and weakened Iran will like...
3    Coronavirus: Iran temporarily frees 54,000 pri...
4    @dassakaye Not necessarily to be honest. #Saud...
Name: 1, dtype: object

In [221]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(pr[1])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [222]:
df.head()

,000,00am,00q3870mh1,01,0120kg,03,035mi,03am,075,083,...,全く同感なのだが,同時にトランプ政権はきっとこの議論を理解しないだろうとも確信している,新型コロナウィルスは米国の対イラン政策の転換のチャンスという記事とその紹介連ツイ,欧州が同航空機の空域入域を拒否したからとのこと,동해상으로,미상,발사,발사체,북한,합참
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.182557,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [223]:
len(posts)

2753

In [224]:
df.iloc[0,:].max()

0.37232215252135864

In [225]:
words = []
for i in range(len(df)):
    words.append(df.iloc[i,:][df.iloc[i,:]==df.iloc[i,:].max()].index.values.tolist())

In [226]:
len(words)

2753

In [227]:
pr['words'] = words

In [228]:
pr.head()

,0,1,words
0,1235946337331343366,@dassakaye Exactly. Never has made it more cle...,[dassakaye]
1,1235944924853317632,LEBANON #Coronavirus Crisis: #COVID19 spreadi...,[lebanon]
2,1235938139119562754,A further isolated and weakened Iran will like...,"[emboldening, isolated, pscktckw8m]"
3,1235161759423614977,"Coronavirus: Iran temporarily frees 54,000 pri...","[ixktbh9qen, unconditional]"
4,1234901226807939072,@dassakaye Not necessarily to be honest. #Saud...,"[managem, necessarily, ph93rk6s04]"


In [229]:
for i in range(len(pr)):
    try:
        posts[pr.loc[i,0]]['words'] = pr.loc[i,'words']
    except:
        continue


In [230]:
#posts

# Add posts to opinion leader

In [231]:
for key, value in posts.items():
    try:
        leaders[value['leader_twitter_id']]["posts"].append(key)
    except:
        continue

In [232]:
leaders[2696666707]

{'_id': ObjectId('5e4fe236f3d95c3a7b5f3b9f'),
 'full_name': 'Dalia Dassa Kaye',
 'twitter_id': 2696666707,
 'twitter_screen_name': 'dassakaye',
 'twitter_created_at': datetime.datetime(2014, 7, 31, 21, 24, 12),
 'twitter_description': 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation',
 'twitter_followers_count': 3489,
 'twitter_friends_count': 403,
 'twitter_location': 'Santa Monica, CA',
 'new_leader': False,
 'twitter_statuses_count': 3660,
 'level_of_certainty': 2,
 'twitter_profile_image': 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg',
 'community_following': [30181014,
  22803302,
  2759399473,
  1227326168,
  2190601093,
  43380129,
  1515812534,
  482120066,
  2739262472,
  1387074108,
  376435807,
  49702592,
  42176166,
  47156708,
  475454826,
  48906364,
  480678403,
  1114663532,
  488772327,
  61051322,
  541858751],
 'lock': False,
 'posts': [1235946337331343366,
  1235944924853317632,
  123593813911956

# LDA

In [233]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

In [234]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [235]:
import spacy

In [236]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [237]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [238]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [239]:
def LDA(posts_corpus):
    df = pd.DataFrame(posts_corpus) 
    df = df.drop_duplicates()
    df = df.rename({0: 'leader', 1: 'post'}, axis='columns')
    
    data = df.post.values.tolist()
    data = [re.sub('\s+', ' ', sent) for sent in data]
    data = [re.sub("\'", "", sent) for sent in data]
    data_words = list(sent_to_words(data))
    
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])


    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=id2word,
                                               num_topics=5, 
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha='auto',
                                               per_word_topics=True)
    return lda_model

In [240]:
for key, value in leaders.items():
    posts_corpus = []
    try:   
        for post in value['posts']:
            leader_posts = []
            leader_posts.append(key)
            leader_posts.append(text(posts[post]))
            posts_corpus.append(leader_posts)
        leaders[key]['topics'] = LDA(posts_corpus).print_topics()
    except:
        continue

In [241]:
leaders[2696666707]

{'_id': ObjectId('5e4fe236f3d95c3a7b5f3b9f'),
 'full_name': 'Dalia Dassa Kaye',
 'twitter_id': 2696666707,
 'twitter_screen_name': 'dassakaye',
 'twitter_created_at': datetime.datetime(2014, 7, 31, 21, 24, 12),
 'twitter_description': 'Director of the Center for Middle East Public Policy (CMEPP) at the RAND Corporation',
 'twitter_followers_count': 3489,
 'twitter_friends_count': 403,
 'twitter_location': 'Santa Monica, CA',
 'new_leader': False,
 'twitter_statuses_count': 3660,
 'level_of_certainty': 2,
 'twitter_profile_image': 'http://pbs.twimg.com/profile_images/494962622597300224/QeFwiaKH_400x400.jpeg',
 'community_following': [30181014,
  22803302,
  2759399473,
  1227326168,
  2190601093,
  43380129,
  1515812534,
  482120066,
  2739262472,
  1387074108,
  376435807,
  49702592,
  42176166,
  47156708,
  475454826,
  48906364,
  480678403,
  1114663532,
  488772327,
  61051322,
  541858751],
 'lock': False,
 'posts': [1235946337331343366,
  1235944924853317632,
  123593813911956

In [242]:
# Compute Perplexity
# print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

In [243]:

# Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

# Neo4j

In [50]:
# len(G)

In [51]:
# G.nodes

In [52]:
# list(G.nodes(data=True))

In [53]:
# list(G.edges())

In [54]:
# import matplotlib.pyplot as plt

In [55]:
# Gr = nx.cycle_graph(G)
# pos = nx.circular_layout(Gr)

# # nx.draw_networkx_nodes(G, pos, node_size=150, node_color='blue', alpha=0.3)
# # nx.draw_networkx_edges(G, pos, width=2, alpha=0.3, edge_color='green')
# # nx.draw_networkx_labels(G, pos, font_size=12, font_family='sans-serif')
# # # # default
# # plt.figure(1)
# # smaller nodes and fonts
# # plt.figure(2)
# # larger figure size
# # nx.draw(G,pos,node_size=60,font_size=8) 
# plt.figure(figsize=(12,12)) 
# nx.draw(G,pos,node_size=60)
# plt.show()
# # # plt.savefig('data.png')  

In [56]:
# print(nx.is_strongly_connected(G))

In [57]:
# isolated = nx.isolates(G)

In [58]:
# for i in isolated:
#     G.remove_node(i)

In [59]:
# nx.center(G)

In [60]:
# nx.periphery(G)

In [61]:
# degree_centrality = nx.degree_centrality(G)

In [62]:
# degree_centrality

In [63]:
# centrality_values = []
# for key, value in degree_centrality.items():
#     centrality_values.append([key, value])

In [64]:
# centrality_values

In [65]:
# centrality_values.sort(key=lambda x: x[1], reverse=True)

In [66]:
# centrality_values

In [67]:
# for leader in centrality_values[:5]:
#     print(leaders[leader[0]])

In [68]:
# for leader in centrality_values[-5:]:
#     print(leaders[leader[0]])

# Graph Neo4j

In [154]:
from py2neo import Graph, Node, Relationship
# from bottle import get, run, request, response, static_file

In [145]:
graph = Graph(password = pwd)

In [199]:
for key, value in leaders.items():
    graph.create(Node("Leader",
                id=key,
                full_name=value['full_name'],
                twitter_screen_name=value['twitter_screen_name'],
                twitter_created_at=value["twitter_created_at"],
                twitter_description=value["twitter_description"]))

In [ ]:
for key, value in leaders.items():
    try:
        existing_u1 = graph.nodes.match("Leader", id=key).first()
        for follower in value["community_following"]:
            existing_u2 = graph.nodes.match("Leader", id=follower).first()
            try:
                graph.create(Relationship(existing_u1, 'FOLLOWS', existing_u2))
            except:
                print(key, follower)
    except:
        continue

    

In [246]:
for key, value in posts.items():
    graph.create(Node("Post",
                id=key,
                full_text=value['full_text'],
                date_created=value['date_created'],
                words=value["words"]))

In [247]:
for key, value in leaders.items():
    try:
        existing_u1 = graph.nodes.match("Leader", id=key).first()
        for post in value["posts"]:
            existing_u2 = graph.nodes.match("Post", id=post).first()
            try:
                graph.create(Relationship(existing_u1, 'TWEETED', existing_u2))
            except:
                print(key, post)
    except:
        continue


In [248]:
# RETWEETED???